# Advanced Linear Regression Model für Bäckerei Umsatzvorhersage

Dieses Notebook implementiert ein erweitertes Regressionsmodell, das für jede Warengruppe ein separates LinearRegression-Modell trainiert. Dies ermöglicht präzisere Vorhersagen durch spezifische Modellierung der Zusammenhänge pro Produktkategorie.

## Aufgabenstellung:
- **Datensatz**: bakery_training_dataset.csv
- **Ziel**: Separate Vorhersagemodelle für jede Warengruppe (1-6)
- **Zeitraum**: 
  - Training: 2013-07-01 bis 2017-07-31
  - Validation: 2017-08-01 bis 2018-07-31
  - Test: 2018-08-01 bis 2019-07-31
- **Features**: Temperatur, Windgeschwindigkeit, Bewölkung, Zeitvariablen, Dummies für Feiertage/Wochentage/Jahreszeiten

In [1]:
# 1. Import der erforderlichen Libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

print("Libraries erfolgreich importiert!")

Libraries erfolgreich importiert!


## 📥 1. Daten einlesen & Aufbereitung

In [2]:
# Datensatz einlesen
dataset_path = '/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv'
print(f"Lade Datensatz von: {dataset_path}")

# Prüfe, ob der Datensatz existiert
if os.path.exists(dataset_path):
    print(f"✅ Datei gefunden!")
    df = pd.read_csv(dataset_path)
    # Validiere Datensatz
    expected_columns = ['Datum', 'Umsatz', 'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Warengruppe']
    missing_cols = [col for col in expected_columns if col not in df.columns]
    if missing_cols:
        print(f"⚠️ Warnung: Folgende erwartete Spalten fehlen: {missing_cols}")
    else:
        print(f"✅ Alle erwarteten Spalten vorhanden")
else:
    raise FileNotFoundError(f"Datei nicht gefunden: {dataset_path}")

# Datum als datetime parsen
df['Datum'] = pd.to_datetime(df['Datum'])

# Überprüfe Warengruppen
if 'Warengruppe' in df.columns:
    warengruppen = df['Warengruppe'].unique()
    print(f"Gefundene Warengruppen: {sorted(warengruppen)}")
elif 'Warengruppe_Nr' in df.columns:
    warengruppen = df['Warengruppe_Nr'].unique()
    print(f"Gefundene Warengruppen (numerisch): {sorted(warengruppen)}")
else:
    # Prüfe auf One-Hot-Encoded Warengruppen
    warengruppen_cols = [col for col in df.columns if col.startswith('Warengruppe_')]
    if warengruppen_cols:
        print(f"Gefundene One-Hot-Encoded Warengruppen: {warengruppen_cols}")
    else:
        print("⚠️ Keine Warengruppen-Spalten gefunden!")

print(f"Datensatz Shape: {df.shape}")
print(f"Datum Range: {df['Datum'].min()} bis {df['Datum'].max()}")
print(f"\nSpalten: {list(df.columns)}")
print(f"\nErste 3 Zeilen:")
print(df.head(3))
print(f"\nFehlende Werte:\n{df.isnull().sum().sort_values(ascending=False)[:10]}")  # Top 10 mit fehlenden Werten

Lade Datensatz von: /workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv
✅ Datei gefunden!
✅ Alle erwarteten Spalten vorhanden
Gefundene Warengruppen: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]
Datensatz Shape: (9334, 34)
Datum Range: 2013-07-01 00:00:00 bis 2018-07-31 00:00:00

Spalten: ['Datum', 'Jahr', 'Monat', 'Tag', 'Wochentag', 'Wochentag_Nr', 'Warengruppe', 'Warengruppe_Name', 'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Wettercode_fehlt', 'ist_feiertag', 'Jahreszeit', 'ist_kiwo', 'Umsatz', 'Warengruppe_Brot', 'Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot', 'Jahreszeit_Winter', 'Jahreszeit_Frühling', 'Jahreszeit_Sommer', 'Jahreszeit_Herbst', 'Wochentag_Nr.1', 'Wochentag_Monday', 'Wochentag_Tuesday', 'Wochentag_Wednesday', 'Wochentag_Thursday', 'Wochentag_Friday', 'Wochentag_Saturday', 'Wochentag_Sunday']

Erste 3 Zeilen:
       Datum  Jahr  

In [3]:
# Warengruppen-Mapping erstellen
# Prüfe, ob eine numerische oder String-basierte Warengruppen-Spalte existiert
if 'Warengruppe_Nr' in df.columns:
    # Numerische Warengruppe ist bereits vorhanden
    warengruppe_col = 'Warengruppe_Nr'
elif 'Warengruppe' in df.columns:
    # String-basierte Warengruppe ist vorhanden - prüfen, ob es numerisch konvertierbar ist
    try:
        # Versuche zu konvertieren, falls es sich um Zahlen als Strings handelt
        df['Warengruppe_Nr'] = pd.to_numeric(df['Warengruppe'])
        warengruppe_col = 'Warengruppe_Nr'
    except:
        # Falls Strings wie "Brot", "Brötchen", etc.
        warengruppe_mapping = {
            'Brot': 1,
            'Brötchen': 2,
            'Croissant': 3,
            'Konditorei': 4,
            'Kuchen': 5,
            'Sonstige': 6
        }
        df['Warengruppe_Nr'] = df['Warengruppe'].map(warengruppe_mapping)
        warengruppe_col = 'Warengruppe_Nr'
else:
    # Wenn nur One-Hot-Encoding vorliegt, konvertiere zurück
    warengruppen_cols = [col for col in df.columns if col.startswith('Warengruppe_')]
    if warengruppen_cols:
        # Mapping für Rückkonvertierung
        warengruppen_mapping = {}
        for i, col in enumerate(warengruppen_cols, 1):
            warenname = col.split('_')[1]
            warengruppen_mapping[col] = (i, warenname)
        
        # Erstelle Warengruppe_Nr basierend auf One-Hot-Encoding
        df['Warengruppe_Nr'] = 0
        df['Warengruppe'] = ''
        
        for col, (nr, name) in warengruppen_mapping.items():
            mask = df[col] == 1
            df.loc[mask, 'Warengruppe_Nr'] = nr
            df.loc[mask, 'Warengruppe'] = name
        
        warengruppe_col = 'Warengruppe_Nr'
    else:
        raise ValueError("Keine Warengruppen-Information im Datensatz gefunden!")

# Überprüfe das Ergebnis
print(f"Verwende Warengruppen-Spalte: {warengruppe_col}")
print(f"Verfügbare Warengruppen: {sorted(df[warengruppe_col].unique())}")

# Erstelle Mapping für Namen und Nummern
warengruppe_num_to_name = {
    1: 'Brot',
    2: 'Brötchen', 
    3: 'Croissant',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Sonstige'
}

# Stelle sicher, dass wir sowohl Nr als auch Namen haben
if 'Warengruppe' not in df.columns:
    df['Warengruppe'] = df['Warengruppe_Nr'].map(warengruppe_num_to_name)

# Zeige einige Daten an
print("\nBeispieldaten mit Warengruppen:")
print(df[['Datum', 'Warengruppe_Nr', 'Warengruppe', 'Umsatz']].head())

# Verteilung der Warengruppen
warengruppen_counts = df['Warengruppe'].value_counts()
print("\nVerteilung der Warengruppen:")
print(warengruppen_counts)

# Zeitbasierter Split vorbereiten
train_period = (df['Datum'] >= '2013-07-01') & (df['Datum'] <= '2017-07-31')
val_period = (df['Datum'] >= '2017-08-01') & (df['Datum'] <= '2018-07-31')
test_period = (df['Datum'] >= '2018-08-01') & (df['Datum'] <= '2019-07-31')

# Kurze Übersicht über die Datensatzaufteilung
print("\nZeitbasierte Aufteilung:")
print(f"Training: {df[train_period].shape[0]} Samples ({df[train_period]['Datum'].min()} bis {df[train_period]['Datum'].max()})")
print(f"Validation: {df[val_period].shape[0]} Samples ({df[val_period]['Datum'].min()} bis {df[val_period]['Datum'].max()})")
print(f"Test: {df[test_period].shape[0]} Samples ({df[test_period]['Datum'].min()} bis {df[test_period]['Datum'].max()})")

Verwende Warengruppen-Spalte: Warengruppe_Nr
Verfügbare Warengruppen: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]

Beispieldaten mit Warengruppen:
       Datum  Warengruppe_Nr  Warengruppe      Umsatz
0 2013-07-01               1            1  148.828353
1 2013-07-02               1            1  159.793757
2 2013-07-03               1            1  111.885594
3 2013-07-04               1            1  168.864941
4 2013-07-05               1            1  171.280754

Verteilung der Warengruppen:
Warengruppe
1    1819
2    1819
3    1819
5    1819
4    1766
6     292
Name: count, dtype: int64

Zeitbasierte Aufteilung:
Training: 7493 Samples (2013-07-01 00:00:00 bis 2017-07-31 00:00:00)
Validation: 1841 Samples (2017-08-01 00:00:00 bis 2018-07-31 00:00:00)
Test: 0 Samples (NaT bis NaT)

Verfügbare Warengruppen: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]

Beispieldaten mit Warengruppen:
       Datum  Warengruppe_Nr  Wa

## 🧠 2. Feature Engineering & Interaktionsterme

In [4]:
# Feature-Auswahl gemäß Anforderung
basic_features = [
    # Numerische Features
    'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung',
    'Jahr', 'Monat', 'Tag', 'Wochentag_Nr',
    # Binäre Features
    'Wettercode_fehlt', 'ist_feiertag', 'ist_kiwo',
]

# Prüfe, ob die grundlegenden Features existieren, ansonsten erzeuge sie
# Zeit-Features
if 'Jahr' not in df.columns:
    df['Jahr'] = df['Datum'].dt.year
if 'Monat' not in df.columns:
    df['Monat'] = df['Datum'].dt.month
if 'Tag' not in df.columns:
    df['Tag'] = df['Datum'].dt.day
if 'Wochentag_Nr' not in df.columns:
    df['Wochentag_Nr'] = df['Datum'].dt.dayofweek + 1  # 1-7 für Montag-Sonntag

# Wetterdaten-Features prüfen
for feature in ['Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Wettercode_fehlt', 'ist_feiertag', 'ist_kiwo']:
    if feature not in df.columns:
        print(f"⚠️ Feature '{feature}' nicht vorhanden - wird mit 0 aufgefüllt")
        df[feature] = 0

# Jahreszeit-Features
jahreszeiten = [
    ('Winter', 0, 2),
    ('Frühling', 3, 5),
    ('Sommer', 6, 8),
    ('Herbst', 9, 11)
]
for jahreszeit, start_monat, end_monat in jahreszeiten:
    # Jahreszeit als numerische Variable
    df[f'Jahreszeit_{jahreszeit}'] = 0
    # Saisonale Dummy-Variablen
    df[f'Jahreszeit_{jahreszeit}_Start'] = 0
    df[f'Jahreszeit_{jahreszeit}_Mitte'] = 0
    df[f'Jahreszeit_{jahreszeit}_Ende'] = 0
    
    # Weisen Sie Werte basierend auf dem Monat zu
    df.loc[(df['Monat'] >= start_monat) & (df['Monat'] <= end_monat), f'Jahreszeit_{jahreszeit}'] = 1
    df.loc[(df['Monat'] == start_monat), f'Jahreszeit_{jahreszeit}_Start'] = 1
    df.loc[(df['Monat'] == end_monat), f'Jahreszeit_{jahreszeit}_Ende'] = 1
    df.loc[(df['Monat'] == (start_monat + end_monat) // 2), f'Jahreszeit_{jahreszeit}_Mitte'] = 1

# Wochentag-Features
wochentage = [
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
]
for wochentag in wochentage:
    df[f'Wochentag_{wochentag}'] = 0
    df.loc[df['Wochentag_Nr'] == (wochentage.index(wochentag) + 1) % 7 + 1, f'Wochentag_{wochentag}'] = 1

# Optionale Features
if 'preisindex' in df.columns:
    basic_features.append('preisindex')
if 'preis_delta' in df.columns:
    basic_features.append('preis_delta')

# Feiertags-Intensität: Tage vor/nach Feiertag
df['tage_vor_feiertag'] = 0
df['tage_nach_feiertag'] = 0
feiertage = df.loc[df['ist_feiertag'] == 1, 'Datum']
for feiertag in feiertage:
    for i in range(1, 4):
        mask_vor = df['Datum'] == (feiertag - pd.Timedelta(days=i))
        mask_nach = df['Datum'] == (feiertag + pd.Timedelta(days=i))
        df.loc[mask_vor, 'tage_vor_feiertag'] = i
        df.loc[mask_nach, 'tage_nach_feiertag'] = i

# Trend-Features
df['laufende_woche'] = (df['Datum'] - df['Datum'].min()).dt.days // 7
df['laufender_monat'] = (df['Datum'].dt.year - df['Datum'].dt.year.min()) * 12 + (df['Datum'].dt.month - 1)

# Interaktionsterme: Warengruppe + Wochentag, Warengruppe + Temperatur, Wetter × Jahreszeit
for warengruppe_nr in sorted(df['Warengruppe_Nr'].unique()):
    warengruppe_name = warengruppe_num_to_name[warengruppe_nr]
    # Interaktion Warengruppe × Temperatur
    df[f'Interaktion_{warengruppe_name}_Temperatur'] = (df['Warengruppe_Nr'] == warengruppe_nr) * df['Temperatur']
    # Interaktion Warengruppe × Wochentag
    for wochentag in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
        if f'Wochentag_{wochentag}' in df.columns:
            df[f'Interaktion_{warengruppe_name}_{wochentag}'] = (df['Warengruppe_Nr'] == warengruppe_nr) * df[f'Wochentag_{wochentag}']

# Interaktion Wetter × Jahreszeit
for jahreszeit in ['Winter', 'Sommer', 'Herbst']:
    if f'Jahreszeit_{jahreszeit}' in df.columns:
        df[f'Interaktion_Temperatur_{jahreszeit}'] = df['Temperatur'] * df[f'Jahreszeit_{jahreszeit}']
        df[f'Interaktion_Bewoelkung_{jahreszeit}'] = df['Bewoelkung'] * df[f'Jahreszeit_{jahreszeit}']

# Feature-Liste erweitern
extended_features = basic_features.copy()
jahreszeit_features = [col for col in df.columns if col.startswith('Jahreszeit_')]
wochentag_features = [col for col in df.columns if col.startswith('Wochentag_')]
interaktions_features = [col for col in df.columns if col.startswith('Interaktion_')]
feiertagsintensitaet_features = ['tage_vor_feiertag', 'tage_nach_feiertag']
trend_features = ['laufende_woche', 'laufender_monat']

extended_features.extend(jahreszeit_features)
extended_features.extend(wochentag_features)
extended_features.extend(interaktions_features)
extended_features.extend(feiertagsintensitaet_features)
extended_features.extend(trend_features)

# Erstelle ein DataFrame zur Überprüfung der Features
feature_check = pd.DataFrame({
    'Feature': extended_features,
    'Vorhanden': [feature in df.columns for feature in extended_features],
    'Datentyp': [str(df[feature].dtype) if feature in df.columns else 'N/A' for feature in extended_features],
    'Fehlende Werte': [df[feature].isnull().sum() if feature in df.columns else 'N/A' for feature in extended_features]
})

print("Feature-Übersicht:")
print(feature_check)

# Fehlende Features hinzufügen und mit 0 füllen
missing_features = [feature for feature in extended_features if feature not in df.columns]
if missing_features:
    print(f"\n⚠️ Folgende Features fehlen und werden mit 0 aufgefüllt: {missing_features}")
    for feature in missing_features:
        df[feature] = 0

# Prüfe auf fehlende Werte in den Features
null_counts = df[extended_features].isnull().sum()
features_with_nulls = null_counts[null_counts > 0]
if not features_with_nulls.empty:
    print(f"\n⚠️ Features mit fehlenden Werten:")
    print(features_with_nulls)
    print("Fülle fehlende Werte mit 0...")
    df[extended_features] = df[extended_features].fillna(0)

print(f"\nGesamtzahl der Features für Modellierung: {len(extended_features)}")
print(f"Interaktions-, Feiertagsintensitäts- und Trend-Features wurden erstellt für bessere Modellgenauigkeit")

Feature-Übersicht:
                          Feature  Vorhanden Datentyp  Fehlende Werte
0                      Temperatur       True  float64               0
1             Windgeschwindigkeit       True  float64               0
2                      Bewoelkung       True  float64               0
3                            Jahr       True    int64               0
4                           Monat       True    int64               0
..                            ...        ...      ...             ...
82  Interaktion_Bewoelkung_Herbst       True  float64               0
83              tage_vor_feiertag       True    int64               0
84             tage_nach_feiertag       True    int64               0
85                 laufende_woche       True    int64               0
86                laufender_monat       True    int32               0

[87 rows x 4 columns]

Gesamtzahl der Features für Modellierung: 87
Interaktions-, Feiertagsintensitäts- und Trend-Features wurden erstellt 

## 🔄 3. Trainingsschleife über 6 Warengruppen mit erweiterten Features

In dieser Sektion trainieren wir separate lineare Regressionsmodelle für jede Warengruppe unter Einbeziehung der neuen Features:
- Feiertags-Intensität (Tage vor/nach Feiertagen)
- Trend-Features (laufende Woche/Monat)
- Interaktions-Features (Warengruppe × Wetter, Warengruppe × Wochentag, Wetter × Jahreszeit)

In [ ]:
# Dictionary zur Speicherung der Modelle und Metriken
models = {}
metrics = {}

# Liste aller Warengruppen-Nummern
warengruppen_nummern = sorted(df['Warengruppe_Nr'].unique())

# Funktion zur Anzeige eines Fortschrittsbalkens
def print_progress_bar(iteration, total, prefix='', suffix='', length=50, fill='█'):
    percent = ("{0:.1f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')
    if iteration == total: 
        print()

print(f"Training von {len(warengruppen_nummern)} separaten Modellen mit erweiterten Features...")

# Für jede Warengruppe ein separates Modell trainieren
for i, warengruppe_nr in enumerate(warengruppen_nummern):
    warengruppe_name = warengruppe_num_to_name[warengruppe_nr]
    
    print_progress_bar(i, len(warengruppen_nummern), prefix=f'Modell für {warengruppe_name}:', suffix='Start')
    
    # 1. Filtere die Daten für diese Warengruppe
    warengruppe_mask = df['Warengruppe_Nr'] == warengruppe_nr
    df_warengruppe = df[warengruppe_mask].copy()
    
    # 2. Wähle alle erweiterten Features und Zielvariable
    # Hier nutzen wir die extended_features Liste, die alle neuen Features enthält
    X = df_warengruppe[extended_features]
    y = df_warengruppe['Umsatz']
    
    # 3. Splitte nach Datum
    X_train = X[train_period & warengruppe_mask]
    y_train = y[train_period & warengruppe_mask]
    
    X_val = X[val_period & warengruppe_mask]
    y_val = y[val_period & warengruppe_mask]
    
    X_test = X[test_period & warengruppe_mask]
    y_test = y[test_period & warengruppe_mask]
    
    print_progress_bar(i + 0.3, len(warengruppen_nummern), prefix=f'Modell für {warengruppe_name}:', suffix='Daten vorbereitet')
    
    # 4. Trainiere das Modell mit allen erweiterten Features
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    print_progress_bar(i + 0.6, len(warengruppen_nummern), prefix=f'Modell für {warengruppe_name}:', suffix='Training abgeschlossen')
    
    # 5. Evaluiere auf dem Validierungsset
    y_val_pred = model.predict(X_val)
    
    # Metriken berechnen
    mae = mean_absolute_error(y_val, y_val_pred)
    r2 = r2_score(y_val, y_val_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    # Negative Vorhersagen auf 0 setzen (Umsatz kann nicht negativ sein)
    y_val_pred = np.maximum(y_val_pred, 0)
    
    # Speichere Modell und Metriken
    models[warengruppe_nr] = model
    metrics[warengruppe_nr] = {
        'Name': warengruppe_name,
        'MAE': mae,
        'R2': r2,
        'RMSE': rmse,
        'Training_Samples': len(X_train),
        'Validation_Samples': len(X_val),
        'Test_Samples': len(X_test),
        'Mean_Umsatz_Train': y_train.mean(),
        'Mean_Umsatz_Val': y_val.mean(),
        'Coefficients': dict(zip(extended_features, model.coef_)),
        'Intercept': model.intercept_
    }
    
    print_progress_bar(i + 1.0, len(warengruppen_nummern), prefix=f'Modell für {warengruppe_name}:', suffix='Abgeschlossen')

print("\n✅ Training für alle Warengruppen mit erweiterten Features abgeschlossen!")

## 📊 Analyse der Modellperformance mit erweiterten Features

Nun analysieren wir, wie die erweiterten Features (Feiertagsintensität, Trends und Interaktionen) die Modellperformance beeinflusst haben.

In [ ]:
# Erstelle eine Funktion zur Feature-Wichtigkeitsanalyse
def analyze_feature_importance(model, features, warengruppe_name, top_n=15):
    """Analysiert die Wichtigkeit von Features in einem linearen Modell"""
    # Extrahiere Koeffizienten
    coefs = model.coef_
    
    # Erstelle DataFrame mit Features und ihren Koeffizienten
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Coefficient': coefs,
        'Abs_Coefficient': np.abs(coefs)
    })
    
    # Sortiere nach absoluter Wichtigkeit
    feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False)
    
    # Zeige Top-N Features
    print(f"Top {top_n} wichtigste Features für {warengruppe_name}:")
    print(feature_importance.head(top_n)[['Feature', 'Coefficient']])
    
    # Visualisiere Top-N Features
    plt.figure(figsize=(10, 8))
    colors = ['green' if x > 0 else 'red' for x in feature_importance.head(top_n)['Coefficient']]
    sns.barplot(
        x='Coefficient', 
        y='Feature', 
        data=feature_importance.head(top_n),
        palette=colors
    )
    plt.title(f'Feature-Wichtigkeit für {warengruppe_name}')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return feature_importance

# Analysiere die Wichtigkeit der verschiedenen Feature-Typen für jede Warengruppe
for warengruppe_nr, model in models.items():
    warengruppe_name = metrics[warengruppe_nr]['Name']
    print(f"\n==== Feature-Analyse für {warengruppe_name} ====")
    
    # Analysiere Feature-Wichtigkeit
    importance_df = analyze_feature_importance(model, extended_features, warengruppe_name)
    
    # Gruppiere nach Feature-Typen
    feature_types = {
        'Basis-Features': basic_features,
        'Jahreszeit-Features': jahreszeit_features,
        'Wochentag-Features': wochentag_features,
        'Interaktions-Features': interaktions_features,
        'Feiertagsintensität-Features': feiertagsintensitaet_features,
        'Trend-Features': trend_features
    }
    
    # Berechne durchschnittliche absolute Wichtigkeit pro Feature-Typ
    print("\nDurchschnittliche absolute Feature-Wichtigkeit nach Kategorie:")
    for category, features in feature_types.items():
        # Filtere Features die tatsächlich im Modell verwendet wurden
        valid_features = [f for f in features if f in importance_df['Feature'].values]
        if valid_features:
            avg_importance = importance_df[importance_df['Feature'].isin(valid_features)]['Abs_Coefficient'].mean()
            print(f"{category}: {avg_importance:.6f}")